### PyTorch Workflow
Lets explore an example of PyTorch end-to-end workflow
<br>

In [2]:
what_were_covering = {1: "data (prepare and load)",
                      2: "build model",
                      3: "fitting the model to data (training)",
                      4: "making predictions and evaluating a model (inference)",
                      5: "saving and loading a model",
                      6: "putting it all together"}



In [4]:
from torch import nn # nn contains building all of PyTorchs building blocks for neural networks
# check PyTorch Version
torch.__version__


'2.7.1'

## Data (Preparing and Loading)
Data can be almost anything... in machine learning/
* Excel spreadsheet
* Images of any kind
* Videos
* Audio
* DNA